In [3]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,5,23,2024-07-02 18:44:19
1,大安區,21,17,4,2024-07-02 18:44:18
2,大安區,16,1,15,2024-07-02 18:42:20
3,大安區,11,10,1,2024-07-02 18:43:19
4,大安區,16,6,10,2024-07-02 18:44:19
...,...,...,...,...,...
1425,臺大公館校區,30,15,7,2024-07-02 18:43:20
1426,臺大公館校區,20,3,17,2024-07-02 18:42:20
1427,臺大公館校區,24,3,20,2024-07-02 18:43:20
1428,臺大公館校區,40,2,38,2024-07-02 18:44:19


In [2]:
df['可借數量']

0        9
1        1
2        1
3        3
4        1
        ..
1425     9
1426    19
1427     8
1428    31
1429     5
Name: 可借數量, Length: 1430, dtype: int64

In [5]:
#將數值分入離散區間。
#此函數對於從連續變數到分類變數也很有用。例如，cut可以將年齡轉換為年齡範圍組。支援分入相同數量的分箱或預先指定的分箱陣列。
#pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)

bins = [0,1,4,7,11,100]
labels = ['0輛','1~3輛','4~6輛','7~10輛','10輛以上']

df['可借'] = pd.cut(df['可借數量'],bins=bins,right=False,labels=labels) ##right=False 的意思就是不包含 右邊的數字

df

,行政區域,總數量,可借數量,可還數量,時間,可借
0,大安區,28,5,23,2024-07-02 18:44:19,4~6輛
1,大安區,21,17,4,2024-07-02 18:44:18,10輛以上
2,大安區,16,1,15,2024-07-02 18:42:20,1~3輛
3,大安區,11,10,1,2024-07-02 18:43:19,7~10輛
4,大安區,16,6,10,2024-07-02 18:44:19,4~6輛
...,...,...,...,...,...,...
1425,臺大公館校區,30,15,7,2024-07-02 18:43:20,10輛以上
1426,臺大公館校區,20,3,17,2024-07-02 18:42:20,1~3輛
1427,臺大公館校區,24,3,20,2024-07-02 18:43:20,1~3輛
1428,臺大公館校區,40,2,38,2024-07-02 18:44:19,1~3輛


In [9]:
df1=df.groupby(by=['行政區域','可借'],observed=True)[['可借數量']].count()
#groupby(by=...) 後都會變成index

df1.rename(columns={'可借數量':'站點數量'})

站點數量
行政區域 可借         
中山區  0輛       13
     1~3輛     53
     4~6輛     38
     7~10輛    23
     10輛以上    29
...          ...
萬華區  0輛        8
     1~3輛      8
     4~6輛     18
     7~10輛    16
     10輛以上    30

[65 rows x 1 columns]